In [1]:
import numpy as np
import pandas as pd

In [2]:
training=open("../Desktop/training.txt","r")
validation=open("../Desktop/validation.txt","r")

In [3]:
train_list=training.readlines()
val_list=validation.readlines()

In [4]:
columns=["word1","word2","feature","label"]
training_df=pd.DataFrame(columns=columns)
validation_df=pd.DataFrame(columns=columns)

In [5]:
words=[]

for i in range (len(val_list)):
    w=val_list[i].split(",")
    if w[2][-1]=='?':
        w[2]=w[2][:-1]
    validation_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])
for i in range (len(train_list)):
    w=train_list[i].split(",")
    training_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])

In [6]:
training_df.head()

,word1,word2,feature,label
0,coconut,rice,infested,0
1,ostrich,emu,cloves,0
2,bathtub,toilet,barrier,0
3,pan,skillet,lid,1
4,pumpkin,spinach,snake,0


In [7]:
word_set=set(words)
len(word_set)

2958

In [8]:
word_dic={}
wordvec=open("../Downloads/wordvector/wordvec.txt","r",errors='ignore')
count=0
for line in wordvec:
    w=line.split(" ",1)[0]
    count+=1
#    print (count)
    if w in word_set:
        wv=line.split(" ")
        wv=wv[1:]
        wv[-1]=wv[-1].rstrip('\n')
        wv=[float(i) for i in wv]
        word_dic[w]=wv        
#        print (len(word_dic))


        

In [9]:

from sklearn.decomposition import TruncatedSVD

In [10]:
def compute_pc(X,npc=10):
    svd = TruncatedSVD(n_components=npc, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    return svd.explained_variance_ratio_.sum(), X - X.dot(pc.transpose()).dot(pc)

x = np.array(word_dic['coconut'])
y = np.array(word_dic['infested'])
x2= np.array(word_dic['rice'])
#print (compute_pc(np.array([x,y]), 1))
#print (compute_pc(np.array([x2,y]), 1))

In [11]:
l=[]
cos_sims_1=[]
cos_sims_2=[]
cos_sims_3=[]
cos_sims_4=[]
euc_1=[]
euc_2=[]
euc_3=[]
manh_1=[]
manh_2=[]
manh_3=[]
from scipy.spatial.distance import cosine as cos
from scipy.spatial.distance import euclidean as euc
from scipy.spatial.distance import cityblock as manh
for index,row in training_df.iterrows():
    x1=np.array(word_dic[row[0]])
    x2=np.array(word_dic[row[1]])
    y=np.array(word_dic[row[2]])
    v1, Comps=compute_pc(np.array([x1,x2,y]), 1)
    cos_sims_1.append(cos(x1, x2))
    cos_sims_2.append(cos(x1,y))
    cos_sims_3.append(cos(x2,y))
    cos_sims_4.append(cos(x1,y)/(1e-5+cos(x2,y)))
    euc_1.append(euc(x1,y))
    euc_2.append(euc(x2,y))
    euc_3.append(euc(x1,y)/(1e-5+euc(x2,y)))
    manh_1.append(manh(x1,y))
    manh_2.append(manh(x2,y))
    manh_3.append(manh(x1,y)-manh(x2,y))
#    training_df['cos_sims'] = (Comps[0].dot(Comps[2]), Comps[1].dot(Comps[2]), Comps[0].dot(Comps[1]))
    l.append(v1)

training_df["v1"]=l
training_df["cos(w1,w2)"]=cos_sims_1
training_df["cos(w1,f)"]=cos_sims_2
training_df["cos(w2,f)"]=cos_sims_3
training_df["cos(w1,f)/cos(w2,f)"]=cos_sims_4
training_df["euc(w1,f)"]=euc_1
training_df["euc(w2,f)"]=euc_2
training_df["euc(w1,f)/euc(w2,f)"]=euc_3
training_df["manh(w1,f)"]=manh_1
training_df["manh(w2,f)"]=manh_2
training_df["manh(w1,f)-manh(w2,f)"]=manh_3
training_df[:100]


,word1,word2,feature,label,v1,"cos(w1,w2)","cos(w1,f)","cos(w2,f)","cos(w1,f)/cos(w2,f)","euc(w1,f)","euc(w2,f)","euc(w1,f)/euc(w2,f)","manh(w1,f)","manh(w2,f)","manh(w1,f)-manh(w2,f)"
0,coconut,rice,infested,0,0.173858,0.350304,0.776556,0.788691,0.984601,8.888531,8.996670,0.987979,125.881445,125.502518,0.378927
1,ostrich,emu,cloves,0,0.019013,0.447193,0.826901,0.876629,0.943263,9.902282,10.298461,0.961529,133.584713,139.573696,-5.988983
2,bathtub,toilet,barrier,0,0.131935,0.308351,0.814430,0.734582,1.108683,8.443239,7.996627,1.055849,117.624149,106.973784,10.650364
3,pan,skillet,lid,1,0.075266,0.266692,0.401804,0.566524,0.709231,6.403142,8.187090,0.782101,89.703266,112.682468,-22.979202
4,pumpkin,spinach,snake,0,0.139725,0.413375,0.681086,0.831447,0.819147,7.848612,9.335818,0.840698,107.097366,126.201844,-19.104479
5,crow,crane,flies,1,0.000957,0.633139,0.538428,0.732142,0.735405,6.548432,8.069501,0.811503,90.769129,111.427149,-20.658020
6,jeans,shirt,bug,0,0.239429,0.261077,0.882776,0.820472,1.075924,9.274451,8.504346,1.090553,129.714862,118.172373,11.542489
7,raisin,cranberry,clasp,0,0.200251,0.404522,0.949384,0.948450,1.000974,9.764937,10.066196,0.970071,131.187720,136.274375,-5.086656
8,pillow,bag,shows,0,0.271316,0.491972,0.923094,0.873927,1.056248,8.402963,8.087048,1.039063,114.391198,107.565702,6.825496
9,tie,blouse,jazz,0,0.037662,0.534282,0.878574,0.894210,0.982503,8.996635,9.768102,0.921021,124.330533,133.235748,-8.905215


In [12]:


def remove_pc(X, npc=1):
    """
    Remove the projection on the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: XX[i, :] is the data point after removing its projection
    """
    pc = compute_pc(X, npc)
    if npc==1:
        XX = X.dot(pc.transpose()) * pc
    else:
        XX = X.dot(pc.transpose()).dot(pc)
    return XX


In [48]:
from sklearn import svm
from sklearn import tree
from sklearn import linear_model
X=[]
Y=[]
for index, row in training_df.iterrows():
    X.append(row[4:9])
#    print (row[12:])

    Y.append(row[3])


clf=svm.SVC()
#clf = tree.DecisionTreeClassifier()
#clf = linear_model.LogisticRegression()

clf.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [49]:
l=[]
cos_sims_1=[]
cos_sims_2=[]
cos_sims_3=[]
cos_sims_4=[]
euc_1=[]
euc_2=[]
euc_3=[]
manh_1=[]
manh_2=[]
manh_3=[]
predicted=[]
correct=0
total=0
count_1=0
count_1_right=0
count_all_1=0
for index,row in validation_df.iterrows():
    x1=np.array(word_dic[row[0]])
    x2=np.array(word_dic[row[1]])
    y=np.array(word_dic[row[2]])
    v1, Comps=compute_pc(np.array([x1,x2,y]), 1)
    cos_sims_1.append(cos(x1, x2))
    cos_sims_2.append(cos(x1,y))
    cos_sims_3.append(cos(x2,y))
    cos_sims_4.append(cos(x1,y)/(1e-5+cos(x2,y)))
    euc_1.append(euc(x1,y))
    euc_2.append(euc(x2,y))
    euc_3.append(euc(x1,y)/(1e-5+euc(x2,y)))
    manh_1.append(manh(x1,y))
    manh_2.append(manh(x2,y))
    manh_3.append(manh(x1,y)-manh(x2,y))
    l.append(v1)
    
    

validation_df["v1"]=l
validation_df["cos(w1,w2)"]=cos_sims_1
validation_df["cos(w1,f)"]=cos_sims_2
validation_df["cos(w2,f)"]=cos_sims_3
validation_df["cos(w1,f)/cos(w2,f)"]=cos_sims_4
validation_df["euc(w1,f)"]=euc_1
validation_df["euc(w2,f)"]=euc_2
validation_df["euc(w1,f)/euc(w2,f)"]=euc_3
validation_df["manh(w1,f)"]=manh_1
validation_df["manh(w2,f)"]=manh_2
validation_df["manh(w1,f)-manh(w2,f)"]=manh_3






In [50]:
for index,row in validation_df.iterrows():
    pre=clf.predict([row[4:9]])  
    predicted.append(pre)
    if pre=="1":
        count_1+=1
        if pre==row[3]:
            count_1_right+=1
    if pre==row[3]:
        correct+=1
    if row[3]=="1":
        count_all_1+=1
    total+=1
#validation_df["predicted"]=predicted
accuracy=correct/total
print(count_1)
print(count_1_right)
print(count_all_1)
precision=count_1_right/count_1
recall=count_1_right/count_all_1
print ("accuracy:",accuracy)
print ("precision:",precision)
print ("recall:",recall)
print ("F1:",2*precision*recall/(precision+recall))
#validation_df[:100][["word1","word2","feature","label","predicted"]]



1409
786
1364
accuracy: 0.5587803085966201
precision: 0.5578424414478353
recall: 0.5762463343108505
F1: 0.566895059502344
